Опишем функцию, реализующую Оракул. 

На входе Оракула $|x\rangle|0\rangle$. Оракул преобразует его в $|x\rangle\left|f_{a}(x)\right\rangle$ (где $f(x)=f(x \oplus a)$)

Такую функцию можно реализовать по следующему алгоритму:

1.

$$
|x\rangle|0\rangle \rightarrow|x\rangle|x\rangle
$$

2.


$|x\rangle|x\rangle \rightarrow|x\rangle|x \oplus a\rangle$ если $x_{j}=0$ для наименьшего $j$, для которого $a_{j}=1$, если  $x_{j}\ne0$, то ничего не меняем.



Сам Алгоритм Саймона

Начальное состояние

$|0\rangle^{\otimes n}|0\rangle^{\otimes n}$

Применяем оператор Адамара

$\frac{1}{\sqrt{2^{n}}} \sum_{x \in\{0,1\}^{n}}|x\rangle|0\rangle^{\otimes n}$

Применяем построенный Оракул 

$\frac{1}{\sqrt{2}^{n}} \sum_{x \in\{0,1\}^{n}}|x\rangle|f(x)\rangle$

Проводим измерение второго регистра. Наблюдаемой будет величина $|f(x)\rangle$, на входе которой $x$, что по сути отвечает первому регистру. При этом измеренному значению функции соответсвует как $x$, так и $y=x \oplus a$. Поэтому после измерения первый регистр представим в виде
$$
\left|\psi \right\rangle=\frac{1}{\sqrt{2}}(|x\rangle+|y\rangle)
$$

Применяем оператор Адамара к первому регистру --

$$
\left|\psi \right\rangle=\frac{1}{\sqrt{2^{n+1}}} \sum_{z \in\{0,1\}^{n}}\left[(-1)^{x \cdot z}+(-1)^{y \cdot z}\right]|z\rangle
$$

Как было доказано в предыдущем задании, при измерении мы получим состояние $|z\rangle$ -- знаем соответствующую ему строку $z$, удовлетворяющую условию ортогональности исходной строке

$$
a \cdot z=0(\bmod 2)
$$

Применяя алгоритм несколько раз, мы найдем другие вектора, ортогональные $a$, и таким образом, найдем  само $a$.


In [9]:
import numpy as np
from qiskit import *
from qiskit.visualization import plot_histogram


def oracle(a):
    n = len(a)
    qr = QuantumCircuit(n*2)
    #создание |x>|x>
    for ind in range(n):
        qr.cx(ind, ind+n)  
    #наименьший индекс j в нашем случае = 1    
    for ind in range(n):
        if a[ind] == '1':
            qr.cx(1, ind + n)
    return qr 



a = '111'

n = len(a)
simon_circuit = QuantumCircuit(n*2, n)
simon_circuit.h(0)  
simon_circuit.h(1)  
simon_circuit.h(2)  
#оракул
simon_circuit += oracle(a)
simon_circuit.h(0)  
simon_circuit.h(1)  
simon_circuit.h(2)

simon_circuit.measure(range(n), range(n))
simon_circuit.draw()

┌───┐     ┌───┐                    ┌─┐                
q_0: ┤ H ├──■──┤ H ├────────────────────┤M├────────────────
     ├───┤  │  └───┘                    └╥┘        ┌───┐┌─┐
q_1: ┤ H ├──┼────■─────────■─────────■───╫──────■──┤ H ├┤M├
     ├───┤  │    │         │  ┌───┐  │   ║ ┌─┐  │  └───┘└╥┘
q_2: ┤ H ├──┼────┼────■────┼──┤ H ├──┼───╫─┤M├──┼────────╫─
     └───┘┌─┴─┐  │    │  ┌─┴─┐└───┘  │   ║ └╥┘  │        ║ 
q_3: ─────┤ X ├──┼────┼──┤ X ├───────┼───╫──╫───┼────────╫─
          └───┘┌─┴─┐  │  └───┘     ┌─┴─┐ ║  ║   │        ║ 
q_4: ──────────┤ X ├──┼────────────┤ X ├─╫──╫───┼────────╫─
               └───┘┌─┴─┐          └───┘ ║  ║ ┌─┴─┐      ║ 
q_5: ───────────────┤ X ├────────────────╫──╫─┤ X ├──────╫─
                    └───┘                ║  ║ └───┘      ║ 
c: 3/════════════════════════════════════╩══╩════════════╩═
                                         0  2            1

In [10]:
backend = BasicAer.get_backend('qasm_simulator')
shots = 1024
results = execute(simon_circuit, backend=backend, shots=shots).result()
counts = results.get_counts()
print(counts)

{'101': 270, '110': 249, '011': 242, '000': 263}



как мы видим, все результаты равновероятны, значит, применяя алгоритм 4 раза, мы можем получить каждый раз одно из состояний $000$,
$001$, $101$, $110$. 

Причем они выпадают случайным независимым образом -- вероятность того, что хотя бы 2 вектора будут лнз -- 3/4. 
Вероятность того, 2 вектора будут лнз после того, как запустили алгоритм 4 раза по формуле из лекций:

$n=3$; $x=1$

$q=\left(1-\frac{1}{2^{2+x}}\right)\left(1-\frac{1}{2^{3+x}}\right) \ldots\left(1-\frac{1}{2^{n+x}}\right)$

$q=\left(1-\frac{1}{2^{3}}\right)\left(1-\frac{1}{2^{4}}\right)$

In [11]:
q=(1-1/2**3)*(1-1/2**4)
p=(3/4)**1
print(q)
print(p)

0.8203125
0.75
